# PyCity Schools Analysis

* There are obvious trends towards better performance for smaller schools (~95% passing overall in small-medium schools vs. ~76% passing overall in large schools), and, perhaps counterintuitively, smaller spending on a per-student basis (~95% passing overall for the two smallest budget ranges vs. ~77% passing overall for the two largest budget ranges). However, with this particular data set that just indicates a difference between charter and district schools. Looking at the summary by school there is only one charter school that is in the 'large' school category and only two charter schools that make it into the top half of the spending groups.


* Looking at just the difference between charter and district schools there is a clear difference in both average scores and passing rates for students, with charter schools outperforming the district schools by roughly three points in average reading scores and by more than six points in average math scores leading to overall passing rates of 95% for charter schools and only 74% for district schools.


* Next step for this analysis should look at standard deviations within the data for individual schools. It stands out that although charter schools only perform 3 points higher in reading, their reading pass rate is almost 16 points higher than the district schools and a 6 point edge in average math scores leads to more than 27 points difference in passing rate for math. This indicates a far smaller variance in scores for the charter schools and should be investigated.


* Assuming that the variance is smaller for the charter schools (noting variance should normally be higher for a smaller sample of a truly random population) it then needs to be determined whether this is due to selection effects (higher admittance standards or higher performing students choosing charter over district schools) or whether there are best practices in place at the charter schools that allow them to better assist low performing students, getting more of them to passing grades.


* Finally, it's interesting to note that, for both math and reading, performance tends to stay relatively stable (within a couple points) across all 4 years at all schools. Also, regardless of charter or district, students tend to perform better at reading than at math (although this is much more pronounced in the district scores).

---

In [2]:
# Dependencies and Setup
import pandas as pd
import numpy as np

# File to Load (Remember to Change These)
school_data_to_load = "Resources/schools_complete.csv"
student_data_to_load = "Resources/students_complete.csv"

# Read School and Student Data File and store into Pandas Data Frames
school_data = pd.read_csv(school_data_to_load)
student_data = pd.read_csv(student_data_to_load)

# Combine the data into a single dataset
sd_df = pd.merge(student_data, school_data, how="left", on=["school_name", "school_name"])
sd_df.head()

,Student ID,student_name,gender,grade,school_name,reading_score,math_score,School ID,type,size,budget
0,0,Paul Bradley,M,9th,Huang High School,66,79,0,District,2917,1910635
1,1,Victor Smith,M,12th,Huang High School,94,61,0,District,2917,1910635
2,2,Kevin Rodriguez,M,12th,Huang High School,90,60,0,District,2917,1910635
3,3,Dr. Richard Scott,M,12th,Huang High School,67,58,0,District,2917,1910635
4,4,Bonnie Ray,F,9th,Huang High School,97,84,0,District,2917,1910635


## District Summary

* Calculate the total number of schools

* Calculate the total number of students

* Calculate the total budget

* Calculate the average math score 

* Calculate the average reading score

* Calculate the overall passing rate (overall average score), i.e. (avg. math score + avg. reading score)/2

* Calculate the percentage of students with a passing math score (70 or greater)

* Calculate the percentage of students with a passing reading score (70 or greater)

* Create a dataframe to hold the above results

* Optional: give the displayed data cleaner formatting

In [3]:
# Calculations to make dataframe construction easier
total_students = sd_df["Student ID"].nunique()
passing_math = sd_df.math_score[sd_df.math_score > 69].count() / total_students
passing_reading = sd_df.reading_score[sd_df.reading_score > 69].count() / total_students

# Create summary dataframe
district_summary = pd.DataFrame({'Total Schools': sd_df.school_name.nunique(),
                               'Total Students': total_students,
                               'Total Budget': school_data.budget.sum(),
                               'Average Math Score': sd_df.math_score.mean(),
                               'Average Reading Score': sd_df.reading_score.mean(),
                               #Line below is for calculation mistakenly refered to in instructions as overall pass rate
                               'Average of Reading & Math Scores': (sd_df.math_score.mean()+sd_df.reading_score.mean())/2,
                               '% Passing Math': passing_math*100,
                               '% Passing Reading': passing_reading*100,
                               #This 'Passing Overall' is the correct passing percentage based on formula for other sections.
                               '% Passing Overall':(passing_math + passing_reading)/2*100}, index=['District Summary'])

# Formatting for numbers in columns
district_summary['Total Students'] = district_summary['Total Students'].map("{:,}".format)
district_summary['Total Budget'] = district_summary['Total Budget'].map("${:,.2f}".format)
district_summary['Average Math Score'] = district_summary['Average Math Score'].map("{:.3f}".format)
district_summary['Average Reading Score'] = district_summary['Average Reading Score'].map("{:.3f}".format)
district_summary['Average of Reading & Math Scores'] = district_summary['Average of Reading & Math Scores'].map("{:.3f}".format)
district_summary['% Passing Math'] = district_summary['% Passing Math'].map("{:.3f}%".format)
district_summary['% Passing Reading'] = district_summary['% Passing Reading'].map("{:.3f}%".format)
district_summary['% Passing Overall'] = district_summary['% Passing Overall'].map("{:.3f}%".format)

# Display summary of district data
district_summary

,Total Schools,Total Students,Total Budget,Average Math Score,Average Reading Score,Average of Reading & Math Scores,% Passing Math,% Passing Reading,% Passing Overall
District Summary,15,"39,170","$24,649,428.00",78.985,81.878,80.432,74.981%,85.805%,80.393%


## School Summary

* Create an overview table that summarizes key metrics about each school, including:
  * School Name
  * School Type
  * Total Students
  * Total School Budget
  * Per Student Budget
  * Average Math Score
  * Average Reading Score
  * % Passing Math
  * % Passing Reading
  * Overall Passing Rate (Average of the above two)
  
* Create a dataframe to hold the above results

In [4]:
# Create grouped dataframe to use for cumulative school data calculations
top_df = sd_df.groupby('school_name')

# Calculations to make dataframe creation cleaner
total_students = top_df["Student ID"].count()
pass_math = sd_df[sd_df.math_score > 69].groupby('school_name')['math_score'].count() / total_students
pass_reading = sd_df[sd_df.reading_score > 69].groupby('school_name')['math_score'].count() / total_students

# Create master dataframe with cumulative school data
combined_df = pd.DataFrame({'Total Students': total_students,
                            'Total School Budget': top_df.budget.mean(),
                           'Per Student Budget': top_df.budget.mean()/top_df["Student ID"].count(),
                           'Average Math Score': top_df.math_score.mean(),
                           'Average Reading Score': top_df.reading_score.mean(),
                           '% Passing Math': pass_math*100,
                           '% Passing Reading': pass_reading*100,
                           '% Overall Passing': (pass_math + pass_reading)/2*100})

# Merge master dataframe with the school dataframe to match schools with their type
combined_df = pd.merge(combined_df,school_data[['school_name','type']],on='school_name', how='left').set_index("school_name")

# Remove the title of the index column and rename 'type' to 'School Type'
del combined_df.index.name
combined_df = combined_df.rename(columns={'type':'School Type'})

# Move 'School Type' column next to the index
cols = list(combined_df)
cols.insert(0, cols.pop(cols.index('School Type')))
combined_df = combined_df.loc[:, cols]

# First five rows of combined frame to be used below. No formatting on numbers so they can be used in further calculations.
combined_df.head()

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
Bailey High School,District,4976,3124928,628.0,77.048432,81.033963,66.680064,81.933280,74.306672
Cabrera High School,Charter,1858,1081356,582.0,83.061895,83.975780,94.133477,97.039828,95.586652
Figueroa High School,District,2949,1884411,639.0,76.711767,81.158020,65.988471,80.739234,73.363852
Ford High School,District,2739,1763916,644.0,77.102592,80.746258,68.309602,79.299014,73.804308
Griffin High School,Charter,1468,917500,625.0,83.351499,83.816757,93.392371,97.138965,95.265668


## Top Performing Schools (By Passing Rate)

* Sort and display the top five schools in overall passing rate

In [5]:
# Sort and assign combined dataframe into new DF that can be formatted
top_schools = combined_df.sort_values("% Overall Passing", ascending=False)

# Format numbers
top_schools['Total Students'] = top_schools['Total Students'].map("{:,}".format)
top_schools['Total School Budget'] = top_schools['Total School Budget'].map("${:,.2f}".format)
top_schools['Per Student Budget'] = top_schools['Per Student Budget'].map("${:,.2f}".format)
top_schools['Average Math Score'] = top_schools['Average Math Score'].map("{:.3f}".format)
top_schools['Average Reading Score'] = top_schools['Average Reading Score'].map("{:.3f}".format)
top_schools['% Passing Math'] = top_schools['% Passing Math'].map("{:.3f}%".format)
top_schools['% Passing Reading'] = top_schools['% Passing Reading'].map("{:.3f}%".format)
top_schools['% Overall Passing'] = top_schools['% Overall Passing'].map("{:.3f}%".format)

# Display top five schools
top_schools.head()

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
Cabrera High School,Charter,"1,858","$1,081,356.00",$582.00,83.062,83.976,94.133%,97.040%,95.587%
Thomas High School,Charter,"1,635","$1,043,130.00",$638.00,83.418,83.849,93.272%,97.309%,95.291%
Pena High School,Charter,962,"$585,858.00",$609.00,83.840,84.045,94.595%,95.946%,95.270%
Griffin High School,Charter,"1,468","$917,500.00",$625.00,83.351,83.817,93.392%,97.139%,95.266%
Wilson High School,Charter,"2,283","$1,319,574.00",$578.00,83.274,83.989,93.868%,96.540%,95.204%


## Bottom Performing Schools (By Passing Rate)

* Sort and display the five worst-performing schools

In [6]:
# Sort and assign combined dataframe into new DF that can be formatted
bottom_schools = combined_df.sort_values("% Overall Passing", ascending=True)

# Format numbers
bottom_schools['Total Students'] = bottom_schools['Total Students'].map("{:,}".format)
bottom_schools['Total School Budget'] = bottom_schools['Total School Budget'].map("${:,.2f}".format)
bottom_schools['Per Student Budget'] = bottom_schools['Per Student Budget'].map("${:,.2f}".format)
bottom_schools['Average Math Score'] = bottom_schools['Average Math Score'].map("{:.3f}".format)
bottom_schools['Average Reading Score'] = bottom_schools['Average Reading Score'].map("{:.3f}".format)
bottom_schools['% Passing Math'] = bottom_schools['% Passing Math'].map("{:.3f}%".format)
bottom_schools['% Passing Reading'] = bottom_schools['% Passing Reading'].map("{:.3f}%".format)
bottom_schools['% Overall Passing'] = bottom_schools['% Overall Passing'].map("{:.3f}%".format)

# Display bottom five schools
bottom_schools.head()

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
Rodriguez High School,District,"3,999","$2,547,363.00",$637.00,76.843,80.745,66.367%,80.220%,73.293%
Figueroa High School,District,"2,949","$1,884,411.00",$639.00,76.712,81.158,65.988%,80.739%,73.364%
Huang High School,District,"2,917","$1,910,635.00",$655.00,76.629,81.183,65.684%,81.316%,73.500%
Johnson High School,District,"4,761","$3,094,650.00",$650.00,77.072,80.966,66.058%,81.222%,73.640%
Ford High School,District,"2,739","$1,763,916.00",$644.00,77.103,80.746,68.310%,79.299%,73.804%


## Math Scores by Grade

* Create a table that lists the average Reading Score for students of each grade level (9th, 10th, 11th, 12th) at each school.

  * Create a pandas series for each grade. Hint: use a conditional statement.
  
  * Group each series by school
  
  * Combine the series into a dataframe
  
  * Optional: give the displayed data cleaner formatting

In [7]:
# Create series for math scores for each grade
mscores_ninth = sd_df[sd_df.grade == '9th'].groupby('school_name')['math_score'].mean()
mscores_tenth = sd_df[sd_df.grade == '10th'].groupby('school_name')['math_score'].mean()
mscores_eleventh = sd_df[sd_df.grade == '11th'].groupby('school_name')['math_score'].mean()
mscores_twelth = sd_df[sd_df.grade == '12th'].groupby('school_name')['math_score'].mean()

# Combine series into a single dataframe, assign names with .to_frame function
mscores_combined = pd.concat([mscores_ninth.to_frame(name='9th'), mscores_tenth.to_frame(name='10th'),
                              mscores_eleventh.to_frame(name='11th'), mscores_twelth.to_frame(name='12th')], axis=1)

# Format scores
mscores_combined['9th'] = mscores_combined['9th'].map("{:.3f}".format)
mscores_combined['10th'] = mscores_combined['10th'].map("{:.3f}".format)
mscores_combined['11th'] = mscores_combined['11th'].map("{:.3f}".format)
mscores_combined['12th'] = mscores_combined['12th'].map("{:.3f}".format)

# Remove index name
del mscores_combined.index.name

# Display math scores for all schools by grade
mscores_combined

,9th,10th,11th,12th
Bailey High School,77.084,76.997,77.516,76.492
Cabrera High School,83.095,83.155,82.766,83.277
Figueroa High School,76.403,76.540,76.884,77.151
Ford High School,77.361,77.672,76.918,76.180
Griffin High School,82.044,84.229,83.842,83.356
Hernandez High School,77.438,77.337,77.136,77.187
Holden High School,83.787,83.430,85.000,82.855
Huang High School,77.027,75.909,76.447,77.226
Johnson High School,77.188,76.691,77.492,76.863
Pena High School,83.625,83.372,84.328,84.122


## Reading Score by Grade 

* Perform the same operations as above for reading scores

In [8]:
# Create series for reading scores for each grade
rscores_ninth = sd_df[sd_df.grade == '9th'].groupby('school_name')['reading_score'].mean()
rscores_tenth = sd_df[sd_df.grade == '10th'].groupby('school_name')['reading_score'].mean()
rscores_eleventh = sd_df[sd_df.grade == '11th'].groupby('school_name')['reading_score'].mean()
rscores_twelth = sd_df[sd_df.grade == '12th'].groupby('school_name')['reading_score'].mean()

# Combine series into a single dataframe, assign names with .to_frame function
rscores_combined = pd.concat([rscores_ninth.to_frame(name="9th"), rscores_tenth.to_frame(name='10th'),
                              rscores_eleventh.to_frame(name='11th'), rscores_twelth.to_frame(name='12th')], axis=1)

# Format scores
rscores_combined['9th'] = rscores_combined['9th'].map("{:.3f}".format)
rscores_combined['10th'] = rscores_combined['10th'].map("{:.3f}".format)
rscores_combined['11th'] = rscores_combined['11th'].map("{:.3f}".format)
rscores_combined['12th'] = rscores_combined['12th'].map("{:.3f}".format)

# Remove index name
del rscores_combined.index.name

# Display reading scores for all schools by grade
rscores_combined

,9th,10th,11th,12th
Bailey High School,81.303,80.907,80.946,80.912
Cabrera High School,83.676,84.253,83.788,84.288
Figueroa High School,81.199,81.409,80.640,81.385
Ford High School,80.633,81.263,80.404,80.662
Griffin High School,83.369,83.707,84.288,84.014
Hernandez High School,80.867,80.660,81.396,80.857
Holden High School,83.677,83.325,83.816,84.699
Huang High School,81.290,81.512,81.417,80.306
Johnson High School,81.261,80.773,80.616,81.228
Pena High School,83.807,83.612,84.336,84.591


## Scores by School Spending

* Create a table that breaks down school performances based on average Spending Ranges (Per Student). Use 4 reasonable bins to group school spending. Include in the table each of the following:
  * Average Math Score
  * Average Reading Score
  * % Passing Math
  * % Passing Reading
  * Overall Passing Rate (Average of the above two)

In [9]:
# Used recommended bins for ease of data comparisons
spending_bins = [0, 585, 615, 645, 675]
group_names = ["<$585", "$585-615", "$615-645", "$645-675"]

# Create copy of the combined data and select only necessary rows
spending_results = combined_df.copy()
spending_results = spending_results[['Per Student Budget', 'Average Math Score', 'Average Reading Score',
                               '% Passing Math', '% Passing Reading', '% Overall Passing']]

# Add 'Results by Budget' column, with group name assigned by bin levels
spending_results["Results by Budget"] = pd.cut(spending_results.loc[:,"Per Student Budget"], spending_bins, labels=group_names)

# Remove 'Per Student Budget' column after it is no longer required.
del spending_results['Per Student Budget']

# Group by bins and generate the averages for each column
spending_results = spending_results.groupby(["Results by Budget"]).mean()

# Rename index for clarity
spending_results.index.names = ['Spending Ranges (per Student)']

# Format numbers
spending_results['Average Math Score'] = spending_results['Average Math Score'].map("{:.3f}".format)
spending_results['Average Reading Score'] = spending_results['Average Reading Score'].map("{:.3f}".format)
spending_results['% Passing Math'] = spending_results['% Passing Math'].map("{:.3f}%".format)
spending_results['% Passing Reading'] = spending_results['% Passing Reading'].map("{:.3f}%".format)
spending_results['% Overall Passing'] = spending_results['% Overall Passing'].map("{:.3f}%".format)

# Display dataframe
spending_results

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
Spending Ranges (per Student),,,,,
<$585,83.455,83.934,93.460%,96.611%,95.035%
$585-615,83.600,83.885,94.231%,95.900%,95.066%
$615-645,79.079,81.891,75.668%,86.107%,80.887%
$645-675,76.997,81.028,66.165%,81.134%,73.649%


## Scores by School Size

* Perform the same operations as above, based on school size.

In [10]:
# Used recommended bins for ease of data comparisons
size_bins = [0, 1000, 2000, 5000]
size_group_names = ["Small (<1000)", "Medium (1000-2000)", "Large (2000-5000)"]

# Create copy of the combined data and select only necessary rows
size_results = combined_df.copy()
size_results = size_results[['Total Students', 'Average Math Score', 'Average Reading Score',
                               '% Passing Math', '% Passing Reading', '% Overall Passing']]

# Add 'Results by Size' column, with group name assigned by bin levels
size_results["Results by Size"] = pd.cut(size_results.loc[:,"Total Students"], size_bins, labels=size_group_names)

# Remove 'Total Students' column after it is no longer required.
del size_results['Total Students']

# Group by bins and generate the averages for each column
size_results = size_results.groupby(["Results by Size"]).mean()

# Rename index for clarity
size_results.index.names = ['School Size']

# Format numbers
size_results['Average Math Score'] = size_results['Average Math Score'].map("{:.3f}".format)
size_results['Average Reading Score'] = size_results['Average Reading Score'].map("{:.3f}".format)
size_results['% Passing Math'] = size_results['% Passing Math'].map("{:.3f}%".format)
size_results['% Passing Reading'] = size_results['% Passing Reading'].map("{:.3f}%".format)
size_results['% Overall Passing'] = size_results['% Overall Passing'].map("{:.3f}%".format)

# Display dataframe
size_results

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
School Size,,,,,
Small (<1000),83.822,83.930,93.550%,96.099%,94.825%
Medium (1000-2000),83.375,83.864,93.600%,96.791%,95.195%
Large (2000-5000),77.746,81.344,69.963%,82.767%,76.365%


## Scores by School Type

* Perform the same operations as above, based on school type.

In [11]:
# Create copy of combined dataframe and select necessary rows
type_results = combined_df.copy()
type_results = type_results[['School Type', 'Average Math Score', 'Average Reading Score',
                               '% Passing Math', '% Passing Reading', '% Overall Passing']]

# Group by 'School Type' and generate averages for each column
type_results = type_results.groupby(["School Type"]).mean()

# Format numbers
type_results['Average Math Score'] = type_results['Average Math Score'].map("{:.3f}".format)
type_results['Average Reading Score'] = type_results['Average Reading Score'].map("{:.3f}".format)
type_results['% Passing Math'] = type_results['% Passing Math'].map("{:.3f}%".format)
type_results['% Passing Reading'] = type_results['% Passing Reading'].map("{:.3f}%".format)
type_results['% Overall Passing'] = type_results['% Overall Passing'].map("{:.3f}%".format)

# Display dataframe
type_results

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
School Type,,,,,
Charter,83.474,83.896,93.621%,96.586%,95.104%
District,76.957,80.967,66.548%,80.799%,73.674%
